In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import itertools as ittls
from collections import Counter

In [2]:
folder = r"/home/czhou2/Documents"
X = np.load('/'.join([folder,'train_x_small.pkl']))
y = np.load('/'.join([folder,'train_y_small.pkl']))
print X.shape,y.shape

(500, 784) (500,)


In [6]:
# Training Parameters
learning_rate = 0.001
training_steps = 60000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)



In [7]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

## generate batch index for loop interation
def batch_gen(batch_size,polulation_size=500):
    collection = []
    for _ in ittls.cycle(xrange(polulation_size)):
        collection.append(_)
        if len(collection) >= batch_size:
            yield collection
            collection = []

def mnist_encoder(y):
    encoded_y = np.zeros((len(y),10),dtype=np.float32)
    for index,label in enumerate(y):
        encoded_y[index,label] = 1.0
    return encoded_y

In [8]:
with tf.Graph().as_default():
    # tf Graph input
    input_x = tf.placeholder("float", [None, timesteps, num_input])
    input_y = tf.placeholder("float", [None, num_classes])
    # Define weights
    weights = {
        'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([num_classes]))
    }
    
    logits = RNN(input_x, weights, biases)
    prediction = tf.nn.softmax(logits)

    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=input_y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)

    # Evaluate model (with test logits, for dropout to be disabled)
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(input_y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:

        # Run the initializer
        sess.run(init)
        index_gen = batch_gen(batch_size= batch_size)
        for step in range(1, training_steps+1):
            
#             batch_x, batch_y = mnist.train.next_batch(batch_size)
            index = next(index_gen)
            batch_x = X[index]
            batch_y = mnist_encoder(y[index])
            # Reshape data to get 28 seq of 28 elements
            batch_x = batch_x.reshape((batch_size, timesteps, num_input))
            # Run optimization op (backprop)
            sess.run(train_op, feed_dict={input_x: batch_x, input_y: batch_y})
            if step % display_step == 0 or step == 1:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={input_x: batch_x,
                                                                     input_y: batch_y})
                print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))

        print("Optimization Finished!")

        # Calculate accuracy for 128 mnist test images
        test_len = 128
        test_data = X[:test_len].reshape((-1, timesteps, num_input))
        test_label = mnist_encoder(y[:test_len])
        print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={input_x: test_data, input_y: test_label}))

Step 1, Minibatch Loss= 3.0182, Training Accuracy= 0.070
Step 200, Minibatch Loss= 2.0679, Training Accuracy= 0.289
Step 400, Minibatch Loss= 1.8519, Training Accuracy= 0.445
Step 600, Minibatch Loss= 1.6542, Training Accuracy= 0.539
Step 800, Minibatch Loss= 1.4743, Training Accuracy= 0.586
Step 1000, Minibatch Loss= 1.3991, Training Accuracy= 0.602
Step 1200, Minibatch Loss= 1.3153, Training Accuracy= 0.617
Step 1400, Minibatch Loss= 1.2004, Training Accuracy= 0.672
Step 1600, Minibatch Loss= 1.0698, Training Accuracy= 0.711
Step 1800, Minibatch Loss= 0.8985, Training Accuracy= 0.773
Step 2000, Minibatch Loss= 1.0192, Training Accuracy= 0.727
Step 2200, Minibatch Loss= 0.9730, Training Accuracy= 0.719
Step 2400, Minibatch Loss= 0.9167, Training Accuracy= 0.797
Step 2600, Minibatch Loss= 0.8385, Training Accuracy= 0.781
Step 2800, Minibatch Loss= 0.6746, Training Accuracy= 0.828
Step 3000, Minibatch Loss= 0.8304, Training Accuracy= 0.781
Step 3200, Minibatch Loss= 0.7758, Training Acc

Step 27200, Minibatch Loss= 0.0073, Training Accuracy= 1.000
Step 27400, Minibatch Loss= 0.0065, Training Accuracy= 1.000
Step 27600, Minibatch Loss= 0.0068, Training Accuracy= 1.000
Step 27800, Minibatch Loss= 0.0049, Training Accuracy= 1.000
Step 28000, Minibatch Loss= 0.0061, Training Accuracy= 1.000
Step 28200, Minibatch Loss= 0.0065, Training Accuracy= 1.000
Step 28400, Minibatch Loss= 0.0059, Training Accuracy= 1.000
Step 28600, Minibatch Loss= 0.0062, Training Accuracy= 1.000
Step 28800, Minibatch Loss= 0.0044, Training Accuracy= 1.000
Step 29000, Minibatch Loss= 0.0055, Training Accuracy= 1.000
Step 29200, Minibatch Loss= 0.0059, Training Accuracy= 1.000
Step 29400, Minibatch Loss= 0.0054, Training Accuracy= 1.000
Step 29600, Minibatch Loss= 0.0057, Training Accuracy= 1.000
Step 29800, Minibatch Loss= 0.0040, Training Accuracy= 1.000
Step 30000, Minibatch Loss= 0.0050, Training Accuracy= 1.000
Step 30200, Minibatch Loss= 0.0054, Training Accuracy= 1.000
Step 30400, Minibatch Lo

Step 54200, Minibatch Loss= 0.0015, Training Accuracy= 1.000
Step 54400, Minibatch Loss= 0.0014, Training Accuracy= 1.000
Step 54600, Minibatch Loss= 0.0016, Training Accuracy= 1.000
Step 54800, Minibatch Loss= 0.0010, Training Accuracy= 1.000
Step 55000, Minibatch Loss= 0.0013, Training Accuracy= 1.000
Step 55200, Minibatch Loss= 0.0014, Training Accuracy= 1.000
Step 55400, Minibatch Loss= 0.0014, Training Accuracy= 1.000
Step 55600, Minibatch Loss= 0.0015, Training Accuracy= 1.000
Step 55800, Minibatch Loss= 0.0010, Training Accuracy= 1.000
Step 56000, Minibatch Loss= 0.0012, Training Accuracy= 1.000
Step 56200, Minibatch Loss= 0.0014, Training Accuracy= 1.000
Step 56400, Minibatch Loss= 0.0013, Training Accuracy= 1.000
Step 56600, Minibatch Loss= 0.0015, Training Accuracy= 1.000
Step 56800, Minibatch Loss= 0.0010, Training Accuracy= 1.000
Step 57000, Minibatch Loss= 0.0012, Training Accuracy= 1.000
Step 57200, Minibatch Loss= 0.0013, Training Accuracy= 1.000
Step 57400, Minibatch Lo

In [35]:
class basic_LSTM(object):
    def __init__(self, sess, time_step, input_dim, hidden_size, class_num, batch_size):
        self.time_step = time_step
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.class_num = class_num
        self.batch_size = batch_size
        self.rnn_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size)
        
        self.input_x = tf.placeholder(dtype = tf.float32, shape = [None,time_step,input_dim])
        self.label_y = tf.placeholder(dtype = tf.float32, shape = [None, class_num])
        
        state = self.rnn_cell.zero_state(batch_size = self.batch_size, dtype = tf.float32)
        
        self.output_list = []
        for each_time_step in tf.unstack(self.input_x, time_step, axis=1):
            output, state = self.rnn_cell(each_time_step, state)
            self.output_list.append(output)
        self.softmax_weight = tf.Variable(tf.random_normal([self.hidden_size,self.class_num]))
        self.softmax_bias = tf.Variable(tf.random_normal([self.class_num]))
        logits = tf.matmul(self.output_list[-1],self.softmax_weight) + self.softmax_bias
        
        self.prediction = tf.nn.softmax(logits)
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=self.label_y))
        
        correct_pred = tf.equal(tf.argmax(self.prediction,1),tf.argmax(self.label_y,1))
        self.accuray = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
        
        sess.run(tf.global_variables_initializer())
        self.trained = False
        
    def batch_gen(self,batch_size,polulation_size=500):
        collection = []
        for _ in ittls.cycle(xrange(polulation_size)):
            collection.append(_)
            if len(collection) >= batch_size:
                yield collection
                collection = []
#     def mnist_encoder(self,y):
#         encoded_y = np.zeros((len(y),10),dtype=np.float32)
#         for index,label in enumerate(y):
#             encoded_y[index,label] = 1.0
#         return encoded_y
    def fit(self, sess, x, y, learning_rate = 0.1,iterations=1000):
        """training process: create optimizer and perform training loop"""
        assert x.shape[1:] == self.input_x.shape[1:]
        
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(self.cost)
        
        index_gen = self.batch_gen(batch_size = self.batch_size, polulation_size=x.shape[0])
        
        for step in xrange(1,iterations+1):
            index = next(index_gen)
            feed_x = x[index]
            feed_y = y[index]
            sess.run(train_op,feed_dict={self.input_x:feed_x, self.label_y:feed_y})
            
            if step % 200 or step==1:
                cost,acc = sess.run([self.cost,self.accuray],
                                    feed_dict={self.input_x:feed_x, self.label_y:feed_y})
                print "step:",step,",batch_cost=","{:.4f}".format(cost),",accuray=","{:.3f}".format(acc)
        self.trained = True
        
    def predict(sess, x):
        return y 
with tf.Graph().as_default():
    with tf.Session() as sess:
        x = np.random.random((500,10,15))
        y = np.random.binomial(n=1, p=0.4, size=(500,1))
        encoded_y = np.zeros((len(y),2),dtype=np.float32)
        for index,label in enumerate(y):
            encoded_y[index,label] = 1.0
        print y.shape
        lstm = basic_LSTM(sess=sess, time_step= 10, input_dim = 15, hidden_size = 20, class_num = 2, batch_size = 50)
        lstm.fit(sess=sess, x=x, y=encoded_y, learning_rate = 0.001,iterations=1000)

(500, 1)
step: 1 ,batch_cost= 1.3125 ,accuray= 0.520
step: 2 ,batch_cost= 1.7398 ,accuray= 0.340
step: 3 ,batch_cost= 1.8699 ,accuray= 0.240
step: 4 ,batch_cost= 1.4470 ,accuray= 0.440
step: 5 ,batch_cost= 1.2081 ,accuray= 0.520
step: 6 ,batch_cost= 1.4125 ,accuray= 0.400
step: 7 ,batch_cost= 1.6736 ,accuray= 0.320
step: 8 ,batch_cost= 1.5873 ,accuray= 0.320
step: 9 ,batch_cost= 1.3896 ,accuray= 0.380
step: 10 ,batch_cost= 1.0856 ,accuray= 0.520
step: 11 ,batch_cost= 1.1102 ,accuray= 0.520
step: 12 ,batch_cost= 1.4436 ,accuray= 0.340
step: 13 ,batch_cost= 1.5374 ,accuray= 0.240
step: 14 ,batch_cost= 1.2168 ,accuray= 0.440
step: 15 ,batch_cost= 1.0220 ,accuray= 0.520
step: 16 ,batch_cost= 1.1795 ,accuray= 0.400
step: 17 ,batch_cost= 1.3978 ,accuray= 0.320
step: 18 ,batch_cost= 1.3183 ,accuray= 0.320
step: 19 ,batch_cost= 1.1624 ,accuray= 0.380
step: 20 ,batch_cost= 0.9294 ,accuray= 0.520
step: 21 ,batch_cost= 0.9586 ,accuray= 0.520
step: 22 ,batch_cost= 1.2112 ,accuray= 0.340
step: 23 ,

step: 182 ,batch_cost= 0.6523 ,accuray= 0.700
step: 183 ,batch_cost= 0.5991 ,accuray= 0.740
step: 184 ,batch_cost= 0.7163 ,accuray= 0.520
step: 185 ,batch_cost= 0.7042 ,accuray= 0.540
step: 186 ,batch_cost= 0.6695 ,accuray= 0.560
step: 187 ,batch_cost= 0.6590 ,accuray= 0.680
step: 188 ,batch_cost= 0.6398 ,accuray= 0.620
step: 189 ,batch_cost= 0.6648 ,accuray= 0.640
step: 190 ,batch_cost= 0.7100 ,accuray= 0.480
step: 191 ,batch_cost= 0.7376 ,accuray= 0.420
step: 192 ,batch_cost= 0.6513 ,accuray= 0.680
step: 193 ,batch_cost= 0.5972 ,accuray= 0.740
step: 194 ,batch_cost= 0.7165 ,accuray= 0.520
step: 195 ,batch_cost= 0.7053 ,accuray= 0.540
step: 196 ,batch_cost= 0.6696 ,accuray= 0.560
step: 197 ,batch_cost= 0.6577 ,accuray= 0.700
step: 198 ,batch_cost= 0.6387 ,accuray= 0.620
step: 199 ,batch_cost= 0.6646 ,accuray= 0.640
step: 201 ,batch_cost= 0.7385 ,accuray= 0.420
step: 202 ,batch_cost= 0.6504 ,accuray= 0.680
step: 203 ,batch_cost= 0.5956 ,accuray= 0.740
step: 204 ,batch_cost= 0.7167 ,acc

step: 367 ,batch_cost= 0.6521 ,accuray= 0.680
step: 368 ,batch_cost= 0.6340 ,accuray= 0.660
step: 369 ,batch_cost= 0.6639 ,accuray= 0.620
step: 370 ,batch_cost= 0.7163 ,accuray= 0.480
step: 371 ,batch_cost= 0.7424 ,accuray= 0.440
step: 372 ,batch_cost= 0.6464 ,accuray= 0.680
step: 373 ,batch_cost= 0.5893 ,accuray= 0.760
step: 374 ,batch_cost= 0.7167 ,accuray= 0.520
step: 375 ,batch_cost= 0.7106 ,accuray= 0.480
step: 376 ,batch_cost= 0.6700 ,accuray= 0.560
step: 377 ,batch_cost= 0.6521 ,accuray= 0.680
step: 378 ,batch_cost= 0.6340 ,accuray= 0.660
step: 379 ,batch_cost= 0.6639 ,accuray= 0.620
step: 380 ,batch_cost= 0.7163 ,accuray= 0.480
step: 381 ,batch_cost= 0.7423 ,accuray= 0.440
step: 382 ,batch_cost= 0.6464 ,accuray= 0.680
step: 383 ,batch_cost= 0.5893 ,accuray= 0.760
step: 384 ,batch_cost= 0.7166 ,accuray= 0.520
step: 385 ,batch_cost= 0.7106 ,accuray= 0.480
step: 386 ,batch_cost= 0.6700 ,accuray= 0.560
step: 387 ,batch_cost= 0.6520 ,accuray= 0.680
step: 388 ,batch_cost= 0.6339 ,acc

step: 553 ,batch_cost= 0.5893 ,accuray= 0.760
step: 554 ,batch_cost= 0.7153 ,accuray= 0.520
step: 555 ,batch_cost= 0.7108 ,accuray= 0.500
step: 556 ,batch_cost= 0.6700 ,accuray= 0.560
step: 557 ,batch_cost= 0.6516 ,accuray= 0.660
step: 558 ,batch_cost= 0.6332 ,accuray= 0.660
step: 559 ,batch_cost= 0.6638 ,accuray= 0.620
step: 560 ,batch_cost= 0.7161 ,accuray= 0.480
step: 561 ,batch_cost= 0.7415 ,accuray= 0.440
step: 562 ,batch_cost= 0.6454 ,accuray= 0.680
step: 563 ,batch_cost= 0.5893 ,accuray= 0.760
step: 564 ,batch_cost= 0.7152 ,accuray= 0.520
step: 565 ,batch_cost= 0.7108 ,accuray= 0.500
step: 566 ,batch_cost= 0.6700 ,accuray= 0.560
step: 567 ,batch_cost= 0.6516 ,accuray= 0.660
step: 568 ,batch_cost= 0.6332 ,accuray= 0.660
step: 569 ,batch_cost= 0.6637 ,accuray= 0.620
step: 570 ,batch_cost= 0.7161 ,accuray= 0.480
step: 571 ,batch_cost= 0.7415 ,accuray= 0.440
step: 572 ,batch_cost= 0.6453 ,accuray= 0.680
step: 573 ,batch_cost= 0.5893 ,accuray= 0.760
step: 574 ,batch_cost= 0.7151 ,acc

step: 744 ,batch_cost= 0.7138 ,accuray= 0.520
step: 745 ,batch_cost= 0.7107 ,accuray= 0.500
step: 746 ,batch_cost= 0.6700 ,accuray= 0.560
step: 747 ,batch_cost= 0.6513 ,accuray= 0.640
step: 748 ,batch_cost= 0.6326 ,accuray= 0.660
step: 749 ,batch_cost= 0.6636 ,accuray= 0.620
step: 750 ,batch_cost= 0.7158 ,accuray= 0.480
step: 751 ,batch_cost= 0.7405 ,accuray= 0.440
step: 752 ,batch_cost= 0.6445 ,accuray= 0.680
step: 753 ,batch_cost= 0.5894 ,accuray= 0.760
step: 754 ,batch_cost= 0.7137 ,accuray= 0.520
step: 755 ,batch_cost= 0.7107 ,accuray= 0.500
step: 756 ,batch_cost= 0.6700 ,accuray= 0.560
step: 757 ,batch_cost= 0.6513 ,accuray= 0.640
step: 758 ,batch_cost= 0.6326 ,accuray= 0.660
step: 759 ,batch_cost= 0.6636 ,accuray= 0.620
step: 760 ,batch_cost= 0.7157 ,accuray= 0.480
step: 761 ,batch_cost= 0.7404 ,accuray= 0.440
step: 762 ,batch_cost= 0.6444 ,accuray= 0.680
step: 763 ,batch_cost= 0.5894 ,accuray= 0.760
step: 764 ,batch_cost= 0.7136 ,accuray= 0.520
step: 765 ,batch_cost= 0.7107 ,acc

step: 924 ,batch_cost= 0.7124 ,accuray= 0.520
step: 925 ,batch_cost= 0.7107 ,accuray= 0.500
step: 926 ,batch_cost= 0.6700 ,accuray= 0.560
step: 927 ,batch_cost= 0.6510 ,accuray= 0.640
step: 928 ,batch_cost= 0.6320 ,accuray= 0.660
step: 929 ,batch_cost= 0.6634 ,accuray= 0.620
step: 930 ,batch_cost= 0.7154 ,accuray= 0.480
step: 931 ,batch_cost= 0.7396 ,accuray= 0.440
step: 932 ,batch_cost= 0.6436 ,accuray= 0.680
step: 933 ,batch_cost= 0.5896 ,accuray= 0.760
step: 934 ,batch_cost= 0.7123 ,accuray= 0.520
step: 935 ,batch_cost= 0.7107 ,accuray= 0.500
step: 936 ,batch_cost= 0.6700 ,accuray= 0.560
step: 937 ,batch_cost= 0.6510 ,accuray= 0.640
step: 938 ,batch_cost= 0.6320 ,accuray= 0.660
step: 939 ,batch_cost= 0.6634 ,accuray= 0.620
step: 940 ,batch_cost= 0.7154 ,accuray= 0.480
step: 941 ,batch_cost= 0.7395 ,accuray= 0.440
step: 942 ,batch_cost= 0.6436 ,accuray= 0.680
step: 943 ,batch_cost= 0.5896 ,accuray= 0.760
step: 944 ,batch_cost= 0.7122 ,accuray= 0.520
step: 945 ,batch_cost= 0.7107 ,acc